In [ ]:
# For Google Colab
!pip install nltk transformers scikit-learn
!pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

df = pd.read_csv('/content/IMDB Dataset.csv', on_bad_lines='skip', quoting=3, encoding='utf-8')
df.head(1)


,,,,,,,,,,,,,,,,,,,,,,,,,,review,sentiment
"""One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right",as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence,which set in right from the word GO. Trust me,this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs,sex or violence. Its is hardcore,in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City,an experimental section of the prison where all the cells have glass fronts and face inwards,so privacy is not high on the agenda. Em City is home to many..Aryans,Muslims,gangstas,Latinos,Christians,Italians,Irish and more....so scuffles,death stares,dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences,forget charm,forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal,I couldn't say I was ready for it,but as I watched more,I developed a taste for Oz,and got accustomed to the high levels of graphic violence. Not just violence,but injustice (crooked guards who'll be sold out for a nickel,inmates who'll kill on order and get away with it,well mannered,middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz,you may become comfortable with what is uncom...,positive


In [ ]:
df.shape

(45999, 2)

In [ ]:


df = df[['review', 'sentiment']]

# Convert sentiment to numeric
df['label'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df = df.dropna().reset_index(drop=True)


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Apply VADER
df['vader_score'] = df['review'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
df['vader_pred'] = df['vader_score'].apply(lambda x: 1 if x >= 0 else 0)


In [ ]:
from transformers import pipeline

# Load sentiment classifier pipeline (BERT)
classifier = pipeline("sentiment-analysis")

# Sample predictions (for speed, limit rows in trial)
sample = df['review'][:100].tolist()
bert_preds = classifier(sample)

# Convert BERT predictions to 0/1
bert_labels = [1 if pred['label'] == 'POSITIVE' else 0 for pred in bert_preds]
df.loc[:99, 'bert_pred'] = bert_labels


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Vectorization
tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(df['review'])
y = df['label']

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict
df['lr_pred'] = model.predict(X)


In [ ]:
from sklearn.metrics import accuracy_score

print("✅ VADER Accuracy:", accuracy_score(df['label'], df['vader_pred']))
print("✅ BERT Accuracy:", accuracy_score(df.loc[:99, 'label'], df.loc[:99, 'bert_pred']))
print("✅ Logistic Regression Accuracy:", accuracy_score(df['label'], df['lr_pred']))


✅ VADER Accuracy: 0.6134453781512605
✅ BERT Accuracy: 0.68
✅ Logistic Regression Accuracy: 0.876750700280112
